# 宝石的识别

In [8]:
import os
import zipfile
import random
import json
import cv2
import numpy as np
from PIL import Image
from matplotlib.pyplot import plot
import paddle
from paddle.io import Dataset
import paddle.nn as nn

# 1.数据准备

In [9]:
# 超参数
train_parameters = {
    # 输入图像的 shape
    "input_size": [3, 224, 224],
    # 分类数
    "class_dim": 25,
    # 原始数据集路径
    "src_path": r"E:\Python\BaiDuPaddle\宝石分类\data\archive_train.zip",
    # 解压目标路径
    "target_path": r"E:\Python\BaiDuPaddle\宝石分类\data\dataset",
    # train_data.txt 路径
    "train_list_path": "./train.txt",
    # eval_data.txt 路径
    "eval_list_path": "./eval.txt",
    # 标签字典
    "label_dict": {},
    # readme_json 路径
    "readme_path": r"E:\Python\BaiDuPaddle\宝石分类\data\readme.json",
    # 训练轮数
    "num_epochs": 10,
    # 批次大小
    "train_batch_size": 32,
    # 优化函数相关配置
    "learning_strategy": {
        # 超惨学习率
        "lr": 0.0005
    }
}

In [10]:
def unzip_data(src_path, target_path):
    '''
    解压原始数据集，将src_path路径下的zip包解压至data/dataset目录下
    '''
    if not os.path.isdir(target_path):
        z = zipfile.ZipFile(src_path, 'r')
        z.extractall(path=target_path)
        z.close()
    else:
        print("文件已解压")

In [11]:
def get_data_list(target_path, train_list_path, eval_list_path):
    '''
    生成数据列表
    '''
    #存放所有类别的信息
    class_detail = []
    #获取所有类别保存的文件夹名称
    data_list_path = target_path
    class_dirs = os.listdir(data_list_path)
    if '__MACOSX' in class_dirs:
        class_dirs.remove('__MACOSX')
    # #总的图像数量
    all_class_images = 0
    # #存放类别标签
    class_label = 0
    # #存放类别数目
    class_dim = 0
    # #存储要写进eval.txt和train.txt中的内容
    trainer_list = []
    eval_list = []
    #读取每个类别
    for class_dir in class_dirs:
        if class_dir != ".DS_Store":
            class_dim += 1
            #每个类别的信息
            class_detail_list = {}
            eval_sum = 0
            trainer_sum = 0
            #统计每个类别有多少张图片
            class_sum = 0
            #获取类别路径
            path = os.path.join(data_list_path, class_dir)
            # 获取所有图片
            img_paths = os.listdir(path)
            for img_path in img_paths:  # 遍历文件夹下的每个图片
                if img_path == '.DS_Store':
                    continue
                name_path = os.path.join(path, img_path)  # 每张图片的路径
                if class_sum % 15 == 0:  # 每10张图片取一个做验证数据
                    eval_sum += 1  # eval_sum为测试数据的数目
                    eval_list.append(name_path + "\t%d" % class_label + "\n")
                else:
                    trainer_sum += 1
                    trainer_list.append(name_path + "\t%d" % class_label + "\n")  #trainer_sum测试数据的数目
                class_sum += 1  #每类图片的数目
                all_class_images += 1  #所有类图片的数目

            # 说明的json文件的class_detail数据
            class_detail_list['class_name'] = class_dir  #类别名称
            class_detail_list['class_label'] = class_label  #类别标签
            class_detail_list['class_eval_images'] = eval_sum  #该类数据的测试集数目
            class_detail_list['class_trainer_images'] = trainer_sum  #该类数据的训练集数目
            class_detail.append(class_detail_list)
            #初始化标签列表
            train_parameters['label_dict'][str(class_label)] = class_dir
            class_label += 1

    #初始化分类数
    train_parameters['class_dim'] = class_dim
    print(train_parameters)
    #乱序
    random.shuffle(eval_list)
    with open(eval_list_path, 'a') as f:
        for eval_image in eval_list:
            f.write(eval_image)
            #乱序
    random.shuffle(trainer_list)
    with open(train_list_path, 'a') as f2:
        for train_image in trainer_list:
            f2.write(train_image)

            # 说明的json文件信息
    readjson = {}
    readjson['all_class_name'] = data_list_path  #文件父目录
    readjson['all_class_images'] = all_class_images
    readjson['class_detail'] = class_detail
    jsons = json.dumps(readjson, sort_keys=True, indent=4, separators=(',', ': '))
    with open(train_parameters['readme_path'], 'w') as f:
        f.write(jsons)
    print('生成数据列表完成！')

In [12]:
'''
参数初始化
'''
src_path = train_parameters['src_path']
target_path = train_parameters['target_path']
train_list_path = train_parameters['train_list_path']
eval_list_path = train_parameters['eval_list_path']
batch_size = train_parameters['train_batch_size']
'''
解压原始数据到指定路径
'''
unzip_data(src_path, target_path)

'''
划分训练集与验证集，乱序，生成数据列表
'''
#每次生成数据列表前，首先清空train.txt和eval.txt
with open(train_list_path, 'w') as f:
    f.seek(0)
    f.truncate()
with open(eval_list_path, 'w') as f:
    f.seek(0)
    f.truncate()

#生成数据列表
get_data_list(target_path, train_list_path, eval_list_path)

文件已解压
{'input_size': [3, 224, 224], 'class_dim': 25, 'src_path': 'E:\\Python\\BaiDuPaddle\\宝石分类\\data\\archive_train.zip', 'target_path': 'E:\\Python\\BaiDuPaddle\\宝石分类\\data\\dataset', 'train_list_path': './train.txt', 'eval_list_path': './eval.txt', 'label_dict': {'0': 'Alexandrite', '1': 'Almandine', '2': 'Benitoite', '3': 'Beryl Golden', '4': 'Carnelian', '5': 'Cats Eye', '6': 'Danburite', '7': 'Diamond', '8': 'Emerald', '9': 'Fluorite', '10': 'Garnet Red', '11': 'Hessonite', '12': 'Iolite', '13': 'Jade', '14': 'Kunzite', '15': 'Labradorite', '16': 'Malachite', '17': 'Onyx Black', '18': 'Pearl', '19': 'Quartz Beer', '20': 'Rhodochrosite', '21': 'Sapphire Blue', '22': 'Tanzanite', '23': 'Variscite', '24': 'Zircon'}, 'readme_path': 'E:\\Python\\BaiDuPaddle\\宝石分类\\data\\readme.json', 'num_epochs': 10, 'train_batch_size': 32, 'learning_strategy': {'lr': 0.0005}}
生成数据列表完成！


In [13]:
class Reader(Dataset):
    def __init__(self, data_path, mode='train'):
        """
        数据读取器
        :param data_path: 数据集所在路径
        :param mode: train or eval
        """
        super().__init__()
        self.dara_path = data_path
        self.img_path = []
        self.labels = []

        if mode == "train":
            with open(os.path.join(self.dara_path, "train.txt"), "r", encoding="utf-8") as f:
                self.info = f.readlines()
                for img_info in self.info:
                    img_path, label = img_info.strip().split("\t")
                    self.img_path.append(img_path)
                    self.labels.append(int(label))
        else:
            with open(os.path.join(self.dara_path, "eval.txt"), "r", encoding="utf-8") as f:
                self.info = f.readlines()
                for img_info in self.info:
                    img_path, label = img_info.strip().split("\t")
                    self.img_path.append(img_path)
                    self.labels.append(int(label))

    def __getitem__(self, index):
        img_path = self.img_path[index]
        label = self.labels[index]
        img = Image.open(img_path)
        # 确保 RGB 通道
        if img.mode != 'RGB':
            img = img.convert("RGB")
        # 更改大小
        img = img.resize((224, 224), Image.BILINEAR)
        # 转为 numpy 数组
        img = np.array(img).astype("float32")
        # 将代表通道的维度提前
        # 将数据规划到 0-1
        img = img.transpose((2, 0, 1)) / 255
        # 处理label
        label = np.array([label], dtype="int64")
        return img, label

    def __len__(self):
        return len(self.img_path)

In [15]:
train_dataset = Reader(r"E:\Python\BaiDuPaddle\宝石分类\\", mode="train")
eval_dataset = Reader(r"E:\Python\BaiDuPaddle\宝石分类\\", mode="eval")

train_loader = paddle.io.DataLoader(train_dataset)
eval_loader = paddle.io.DataLoader(eval_dataset)

# 2.定义模型

In [16]:
class MyDNN(nn.Layer):
    def __init__(self):
        super(MyDNN, self).__init__()
        self.linear1 = nn.Linear(224 * 224 * 3, 1024)
        self.linear2 = nn.Linear(1024, 512)
        self.linear3 = nn.Linear(512, 256)
        self.linear4 = nn.Linear(256, 128)
        self.linear5 = nn.Linear(128, 25)
        self.relu1 = nn.ReLU()
        self.relu3 = nn.ReLU()
        self.relu4 = nn.ReLU()

    def forward(self, input):
        x = self.linear1(input)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        x = self.relu4(x)
        x = self.linear5(x)
        return x

In [18]:
from matplotlib import pyplot as plt

Batch=0
Batchs=[]
all_train_accs=[]
def draw_train_acc(Batchs, train_accs):
    title="training accs"
    plt.title(title, fontsize=24)
    plt.xlabel("batch", fontsize=14)
    plt.ylabel("acc", fontsize=14)
    plt.plot(Batchs, train_accs, color='green', label='training accs')
    plt.legend()
    plt.grid()
    plt.show()

all_train_loss=[]
def draw_train_loss(Batchs, train_loss):
    title="training loss"
    plt.title(title, fontsize=24)
    plt.xlabel("batch", fontsize=14)
    plt.ylabel("loss", fontsize=14)
    plt.plot(Batchs, train_loss, color='red', label='training loss')
    plt.legend()
    plt.grid()
    plt.show()

# 3.训练模型

In [19]:
model = MyDNN()
model.train()
cross_entropy = paddle.nn.CrossEntropyLoss()
opt = paddle.optimizer.SGD(learning_rate=train_parameters["learning_strategy"]["lr"], parameters=model.parameters())
epochs_num = train_parameters["num_epochs"]

for epoch in range(epochs_num):
    for batch_id, data in enumerate(train_loader):
        img = data[0]
        label = data[1]
        img = img.flatten(1)
        predict = model(img)
        loss = cross_entropy(predict, label)
        acc = paddle.metric.accuracy(predict, label)
        loss.backward()
        opt.step()
        opt.clear_grad()
        if batch_id != 0 and batch_id % 5 == 0:
            Batch += 5
            Batchs.append(Batch)
            print(f"epoch:{epoch},step{batch_id},train_loss{loss.numpy()[0]}, train_acc{acc.numpy()[0]}")
        paddle.save(model.state_dict(), "MyDNN")

epoch:0,step5,train_loss5.284036159515381, train_acc0.0
epoch:0,step10,train_loss3.5361685752868652, train_acc0.0
epoch:0,step15,train_loss4.816277503967285, train_acc0.0
epoch:0,step20,train_loss3.1913626194000244, train_acc0.0
epoch:0,step25,train_loss2.1728124618530273, train_acc0.0
epoch:0,step30,train_loss3.5143251419067383, train_acc0.0
epoch:0,step35,train_loss4.311332702636719, train_acc0.0
epoch:0,step40,train_loss4.915657043457031, train_acc0.0
epoch:0,step45,train_loss3.639138698577881, train_acc0.0
epoch:0,step50,train_loss3.6179466247558594, train_acc0.0
epoch:0,step55,train_loss3.780313014984131, train_acc0.0
epoch:0,step60,train_loss3.3033974170684814, train_acc0.0
epoch:0,step65,train_loss3.3663084506988525, train_acc0.0
epoch:0,step70,train_loss4.239714622497559, train_acc0.0
epoch:0,step75,train_loss3.251542806625366, train_acc0.0
epoch:0,step80,train_loss3.7533719539642334, train_acc0.0
epoch:0,step85,train_loss6.0252227783203125, train_acc0.0
epoch:0,step90,train_lo

KeyboardInterrupt: 

# 4.模型评估

In [ ]:
para_state_dict = paddle.load("MyDNN")
model = MyDNN()
model.set_state_dict(para_state_dict)
model.eval()
accs = []
for batch_id, data in enumerate(eval_loader):
    img = data[0]
    label = data[0]
    img = img.flatten(1)
    predict = model(img)
    acc = paddle.metric.accuracy(predict, label)
    accs.append(acc.numpy()[0])

avg_acc = np.mean(accs)
print("当前模型在验证集上的准确率为:",avg_acc)